## Meteo forcing example

This is an example of reading locally stored ECMWF grib files. 

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.meteo as pmeteo
import datetime
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import pandas as pd
import numpy as np

In [ ]:
hv.notebook_extension('bokeh')

In [ ]:
pyPoseidon.__version__

### setup

In [ ]:
#define in a dictionary the properties of the model..
dic={'minlon':-35., # lat/lon window
     'maxlon':42.,
     'minlat':25.05,
     'maxlat':76.5,
     'start_date':'1979-2-1',
     'end_date':'1979-2-2'
    }

In [ ]:
# This just a structure to create the list of paths for the actual meteo files - change appropriately 
start_date = pd.to_datetime(dic['start_date']) 
end_date = pd.to_datetime(dic['end_date']) 
#keep the years
dr = np.arange(start_date.year,end_date.year+1)

#creating a sequence of folder from which we read the meteo.
PATH='/Users/brey/DATA/ERAI/'#Path to meteo files
meteo_files = [PATH+'eraInterim_{:04d}.grib'.format(x) for x in dr]
meteo_files

In [ ]:
dic.update({'mpaths':meteo_files})

### retrieve UVP

In [ ]:
#get uvp
meteo = pmeteo.erai(**dic)
#meteo = generic(filename='/Users/brey/Downloads/wrf_uvp.nc')
#meteo = hnms.oper(**dic)

In [ ]:
meteo.uvp

In [ ]:
meteo.uvp.time

## Visualise

### holoviews

In [ ]:
%%opts Image [width=650 height=450] (cmap='jet')
meteo.hview.to(hv.Image, ['longitude','latitude'], 'msl').hist()

### geoviews

In [ ]:
g50 = gf.coastline(plot=dict(scale='50m'), style=dict(linewidth=1.5))

In [ ]:
%%opts Image [colorbar=True width=650 height=450 toolbar="above"  tools=['hover']] (cmap='viridis')
#meteo.gview.to(gv.Image, ['longitude','latitude'], 'msl', dynamic=True) * g50 * gf.borders() # faster
meteo.gview.to(gv.Image, ['longitude','latitude'], 'msl') * g50 * gf.borders()

## Output to file

In [ ]:
#generic netCDF output
meteo.uvp.to_netcdf('./test/test.nc') # to netcdf

In [ ]:
#save to DELFT3D format
meteo.output(solver='d3d',rpath='./test/') # to u,v,p for d3d

In [ ]:
#save to SCHISM format
meteo.output(solver='schism',rpath='./test/')